In [17]:
import os

In [18]:
%pwd

'/home/nikhilds/Projects/Kidney-Disease-Classification/research'

In [19]:
os.chdir("..")

In [20]:
%pwd

'/home/nikhilds/Projects/Kidney-Disease-Classification'

In [21]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str 
    local_data_file : Path
    unzip_dir : Path

In [35]:
from box import ConfigBox
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [36]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->ConfigBox:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [37]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [38]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_data(self):
        try:
            download_URL = self.config.source_URL
            zipfile_path = self.config.local_data_file
            root_dir = self.config.root_dir

            os.makedirs(root_dir,exist_ok=True)
            logger.info(f"Downloading dataset from {download_URL} into {zipfile_path}")

            file_id = download_URL.split('/')[-2]
            prefix_url = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix_url+file_id,zipfile_path)
            logger.info(f"Downloaded dataset from {download_URL} into {zipfile_path}")

        except Exception as e:
            raise e
        
    def extract_data(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [39]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_data()
except Exception as e:
    raise e

[2024-09-29 00:54:34,174: INFO: common: yaml file: config/config.yaml loaded successfully:]
[2024-09-29 00:54:34,175: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-09-29 00:54:34,175: INFO: common: created directory at: artifacts:]
[2024-09-29 00:54:34,176: INFO: common: created directory at: artifacts/data_ingestion:]
[2024-09-29 00:54:34,176: INFO: 3150691259: Downloading dataset from https://drive.google.com/file/d/1EFvdGjOAlQMpd7Zn11JMNxb8QRErLBAa/view?usp=sharing into artifacts/data_ingestion/data.zip:]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1EFvdGjOAlQMpd7Zn11JMNxb8QRErLBAa
From (redirected): https://drive.google.com/uc?export=download&id=1EFvdGjOAlQMpd7Zn11JMNxb8QRErLBAa&confirm=t&uuid=84faef76-c506-4bcc-8b01-71a800102858
To: /home/nikhilds/Projects/Kidney-Disease-Classification/artifacts/data_ingestion/data.zip
100%|██████████| 768M/768M [01:13<00:00, 10.5MB/s] 

[2024-09-29 00:55:58,107: INFO: 3150691259: Downloaded dataset from https://drive.google.com/file/d/1EFvdGjOAlQMpd7Zn11JMNxb8QRErLBAa/view?usp=sharing into artifacts/data_ingestion/data.zip:]
